In [3]:
include("TheroFunc.jl")
include("constants.jl")
include("Tools.jl")
include("Functions_Quark.jl")
include("Phase_Diagram.jl")

using Plots


using DataFrames
using CSV
using BenchmarkTools
using LaTeXStrings
using Peaks
using MeshGrid

In [4]:
using FastGaussQuadrature
using MeshGrid
using QuadGK


In [5]:
function gauleg(a, b, n)
    x, w = gausslegendre(n)
    x_mapped = (b - a) / 2 .* x .+ (b + a) / 2
    w_mapped = (b - a) / 2 .* w
    return x_mapped, w_mapped
end



gauleg (generic function with 1 method)

In [6]:
function get_nodes(p_num, n_num)
    p1, w1 = gauleg(0.0, 15.0, p_num)
    ns = range(0, n_num-1)
    wn = ones(length(ns))


    P1, N1 = meshgrid(p1, ns)
    W1, WN = meshgrid(w1, wn)
    Alpha = ones(n_num)
    for i in 2:n_num
        Alpha[i] = 2
    end
    W = W1 .* WN .* Alpha
    ints = [P1, N1, W]
    return ints
end

get_nodes (generic function with 1 method)

In [7]:
mass1 = 1.0
mass2 = 1.0
mass3 = 2.0

eB = 0.1
qf1 = 2/3
qf2 = 1/3
qf3 = 1/3

0.3333333333333333

In [ ]:


P, N, W = get_nodes(128, 100)

pu = (2 .*N.*qf1.*eB .+ P.^2).^0.5
pd = (2 .*N.*qf2.*eB .+ P.^2).^0.5
ps = (2 .*N.*qf3.*eB .+ P.^2).^0.5

Eu_n = (pu.^2 .+ mass1^2).^0.5 .* Lambda_f^(20) ./(Lambda_f^(20) .+ pu.^(20))
Ed_n = (pd.^2 .+ mass2^2).^0.5 .* Lambda_f^(20) ./(Lambda_f^(20) .+ pd.^(20))
Es_n = (ps.^2 .+ mass3^2).^0.5 .* Lambda_f^(20) ./(Lambda_f^(20) .+ ps.^(20))

Es = sum(W .* (Eu_n .+ Ed_n .+ Es_n))



1.0732037480920553e-10


In [18]:
masses = [mass1, mass2, mass3]
qf = [qf1, qf2, qf3]

masses_col = reshape(masses, 3, 1)
# 计算三种夸克的动量
p_array = [(2 .* N .* qf[i] .* eB .+ P.^2).^0.5 for i in 1:3]

# 计算能量
E = [sqrt.(p.^2 .+ masses_col[i]^2) .* 
        Lambda_f^(20) ./ (Lambda_f^(20) .+ p.^(20)) 
        for (i, p) in enumerate(p_array)]


3-element Vector{Matrix{Float64}}:
 [1.0000008624311267 1.000023936341151 … 2.2461148681919604e-13 2.2302146326869131e-13; 1.0645821049577766 1.0646037791538145 … 2.2334947906799595e-13 2.2176932305924929e-13; … ; 0.12312476293947465 0.12312075925356451 … 1.3130080295808434e-13 1.3042220757357572e-13; 0.11211955787960463 0.11211593695740864 … 1.3060348949632628e-13 1.2973004806946952e-13]
 [1.0000008624311267 1.000023936341151 … 2.2461148681919604e-13 2.2302146326869131e-13; 1.0327963940340148 1.032818735268642 … 2.2397950053906903e-13 2.2239441916920392e-13; … ; 2.667933160202563 2.66793606148851 … 1.7108138818063422e-13 1.6990438593190236e-13; 2.6717280429882577 2.671730388880164 … 1.7061359662210344e-13 1.6944014964393474e-13]
 [2.0000004312157027 2.000011968278003 … 2.260987174535841e-13 2.2449707068105746e-13; 2.0165982226337658 2.016609664739787 … 2.2546211026263713e-13 2.2386544520363634e-13; … ; 3.154744940865584 3.154745619488962 … 1.7218242363909045e-13 1.7099705831276047e-13

In [3]:
# 2025/9/8 测试Pressure 
# 测试Omega函数的脚本
include("TheroFunc.jl")  # 已经包含了constants.jl和tools.jl
using BenchmarkTools

function test_omega()
    # 设置测试参数
    phi = [0.1, 0.1, 0.1]       # 手征凝聚参数 [phi_u, phi_d, phi_s]
    Phi1 = 0.5                  # Polyakov loop 参数
    Phi2 = 0.5                  # 共轭 Polyakov loop 参数
    T = 1.2                     # 温度 (GeV)
    mu_B = 0.0                  # 重子化学势 (GeV)
    eB = 0.1                    # 磁场强度 (GeV^2)
    
    # 获取高斯积分节点
    P, N, W = get_nodes(128, 1)
    gauss_nodes = [P, N, W]
    
    # 计算热力学势
    result = Omega(phi, Phi1, Phi2, T, mu_B, eB, gauss_nodes)

    return result
end
# 运行测试
test_omega()



-0.5492579230794753

In [19]:
# 2025/9/8 测试解方程



include("Phase_Diagram.jl")
include("constants.jl")
include("TheroFunc.jl")


using DataFrames
using CSV

T = 350.0
mu_B = 0.0
P, N, W = get_nodes(64, 101)
gauss_nodes = [P, N, W]
X0 = [-1.5, -1.5, -2.5, 0.54, 0.54]
eB = 0.1 * (1000 / hc)^2
phi = X0[1:3]
Phi1 = X0[4]
Phi2 = X0[5]

res1 = Omega(phi, Phi1, Phi2, T/hc, mu_B, eB, gauss_nodes)
#res = Tmu(T, mu_B, X0, eB, gauss_nodes)
#res2 = function_Quark_core(X0, T/hc, mu_B, eB, gauss_nodes)


-22.18930498349269

chi = 5.224891334506841, term1 = -14.098713772944715, term2=-7.6050014176275615, U = -5.7104811274272524


In [1]:
# 2025/9/8 测试解方程



include("Phase_Diagram.jl")
include("constants.jl")

using DataFrames
using CSV

T = 50.0:2.0:300.0
mu_B = 0.0
P, N, W = get_nodes(128, 100)
gauss_nodes = [P, N, W]
X0 = [-1.8, -1.8, -2.5, 0.01, 0.01]
eB = 0.1 * (1000 / hc)^2


Ts = T 
my_data = DataFrame(T=Float64[], phi_u=Float64[], phi_d=Float64[], phi_s=Float64[], Phi1=Float64[], Phi2=Float64[])

for (i,T) in enumerate(Ts)
    result = Tmu(T, mu_B, X0, eB, gauss_nodes)
    push!(my_data, (T, result...))
    X0 = result  # 使用上一次的结果作为下一次的初始猜测
end
CSV.write("phase_diagram_eB_$(eB*hc^2/1000)_muB_$(mu_B).csv", my_data)


"phase_diagram_eB_100.0_muB_0.0.csv"